In [21]:
# Standard library imports
import argparse
import re

# Third-party libraries
import contractions
import nltk
import numpy as np
import pandas as pd
from joblib import dump, load
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

# NLTK downloads (optional to keep here, can also be run separately)
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

True

## Load data

In [30]:
def load_data(filepath):
    data = pd.read_csv(filepath, sep='\t', names=['title', 'from', 'genre', 'director', 'plot'])
    return data

In [31]:
data = load_data('train.txt')
data.head()

,title,from,genre,director,plot
0,Ela Cheppanu,Telugu,romance,Ramana,Sekhar (Tarun) is a graduate from IIM and work...
1,A Nightmare on Elm Street,American,horror,Samuel Bayer,Kris Fowles (Katie Cassidy) goes to the Spring...
2,American Gothic,American,horror,John Hough,Cynthia is traumatized by the death of her bab...
3,Gang,Bollywood,crime,Mazhar Khan,"Four friends, Gangu (Jackie Shroff), Abdul (Na..."
4,Intimate Relations,British,drama,Charles Frank,Crisis in a middle-class family when the son f...


# Clean the text

In [24]:
# Function to clean the text
def clean_text(text):
    # Check if the text is a non-empty string
    if not isinstance(text, str) or not text.strip():
        return ''

    # Skip applying contractions if the text is too long or complex
    if len(text) > 500:  # Threshold to skip contraction expansion for long texts
        return text

    # Try expanding contractions safely
    try:
        text = contractions.fix(text)
    except Exception as e:
        print(f"Error expanding contractions: {e}, for text: {text[:100]}...")  # Only print the first 100 characters
        return text  # Return the original text if expansion fails

    # Remove c
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    return text

# Function to convert nltk POS tag to wordnet POS tag

In [25]:
# Function to convert nltk POS tag to wordnet POS tag
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ  # Adjective
    elif tag.startswith('V'):
        return wordnet.VERB  # Verb
    elif tag.startswith('N'):
        return wordnet.NOUN  # Noun
    elif tag.startswith('R'):
        return wordnet.ADV  # Adverb
    else:
        return wordnet.NOUN  # Default to noun

# Lemmatize the text

In [36]:
# Function to lemmatize the text
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    # Clean the text
    text = clean_text(text)
    # Tokenize the text
    tokens = nltk.word_tokenize(text.lower())
    # Remove stop words and non-alphabetic tokens
    tokens = [word for word in tokens if word.isalpha() and word]
    # Perform POS tagging
    tagged_tokens = nltk.pos_tag(tokens)
    # Lemmatize each token using the POS tag
    lemmatized_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in tagged_tokens]
    return ' '.join(lemmatized_tokens)


In [37]:
lemmatize_text("unhappy")

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\tester.LAPTOP-NJPROMP8/nltk_data'
    - 'c:\\Users\\tester.LAPTOP-NJPROMP8\\OneDrive - Universidade de Lisboa\\Aaa Técnico\\1. MEIC\\LN\\project\\LN-proj\\.venv\\nltk_data'
    - 'c:\\Users\\tester.LAPTOP-NJPROMP8\\OneDrive - Universidade de Lisboa\\Aaa Técnico\\1. MEIC\\LN\\project\\LN-proj\\.venv\\share\\nltk_data'
    - 'c:\\Users\\tester.LAPTOP-NJPROMP8\\OneDrive - Universidade de Lisboa\\Aaa Técnico\\1. MEIC\\LN\\project\\LN-proj\\.venv\\lib\\nltk_data'
    - 'C:\\Users\\tester.LAPTOP-NJPROMP8\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'


# Preprocess the data

In [27]:
# Preprocess the data
def preprocess_data(data, lemmatize, combine_fields):
    # Extract relevant columns (plot, combine_fields, genre)
    selected_fields = ['plot'] + combine_fields
    if 'genre' in data.columns:
        selected_fields.append('genre')
    data = data[selected_fields].copy()

    # Handle missing values: drop rows with missing 'plot', fill missing combine_fields with ''
    data.dropna(subset=['plot'], inplace=True)  # Ensure 'plot' is not NaN
    for field in combine_fields:
        data[field] = data[field].fillna('')  # Replace NaN in combine_fields with empty strings

    # Combine plot and other specified fields into a single feature
    data['combined_text'] = data['plot']
    for field in combine_fields:
        data['combined_text'] += ' ' + data[field]

    # Apply lemmatization if specified
    if lemmatize:
        data['combined_text'] = data['combined_text'].apply(lemmatize_text)
    else:
        # Clean the text anyway
        data['combined_text'] = data['combined_text'].apply(clean_text)

    return data


In [32]:
data

,title,from,genre,director,plot
0,Ela Cheppanu,Telugu,romance,Ramana,Sekhar (Tarun) is a graduate from IIM and work...
1,A Nightmare on Elm Street,American,horror,Samuel Bayer,Kris Fowles (Katie Cassidy) goes to the Spring...
2,American Gothic,American,horror,John Hough,Cynthia is traumatized by the death of her bab...
3,Gang,Bollywood,crime,Mazhar Khan,"Four friends, Gangu (Jackie Shroff), Abdul (Na..."
4,Intimate Relations,British,drama,Charles Frank,Crisis in a middle-class family when the son f...
...,...,...,...,...,...
8036,Sa 'Yo Lamang,Filipino,drama,Laurice Guillen,Dianne seems to have a happy family. She and h...
8037,The Lemon Sisters,American,drama,Joyce Chopra,Three lifelong friends work the bars in 1980's...
8038,Dil Ne Phir Yaad Kiya,Bollywood,romance,Unknown,"City-based Ashok, who works as a Salesman in a..."
8039,Rehnaa Hai Terre Dil Mein,Bollywood,romance,Gautham Menon,The story starts with narration by Maddy (R. M...


In [35]:
combine_fields = ['from', 'director', 'title']
data = preprocess_data(data, True, combine_fields)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\tester.LAPTOP-NJPROMP8/nltk_data'
    - 'c:\\Users\\tester.LAPTOP-NJPROMP8\\OneDrive - Universidade de Lisboa\\Aaa Técnico\\1. MEIC\\LN\\project\\LN-proj\\.venv\\nltk_data'
    - 'c:\\Users\\tester.LAPTOP-NJPROMP8\\OneDrive - Universidade de Lisboa\\Aaa Técnico\\1. MEIC\\LN\\project\\LN-proj\\.venv\\share\\nltk_data'
    - 'c:\\Users\\tester.LAPTOP-NJPROMP8\\OneDrive - Universidade de Lisboa\\Aaa Técnico\\1. MEIC\\LN\\project\\LN-proj\\.venv\\lib\\nltk_data'
    - 'C:\\Users\\tester.LAPTOP-NJPROMP8\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'


# Split data into training and test sets

In [15]:
# Split data into training and test sets
def split_data(data):
    X = data['combined_text']
    y = data['genre']
    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.1, random_state=42, stratify=y_encoded)
    return X_train, X_test, y_train, y_test, label_encoder


In [16]:
X_train, X_test, y_train, y_test, label_encoder = split_data(data)

# Feature extraction using BOW Vectorizer

In [18]:

# create the vocabulary
vectorizer = CountVectorizer()
vectorizers = {
        "BOW:" 
    }
vectorizer_BOW = CountVectorizer(
    max_features=max_features,
    ngram_range=ngram_range,
    min_df=5)

# Feature extraction using BOW Vectorizer
def extract_features(X_train, X_test, max_features, ngram_range):
    # create the vocabulary
    
    X_train_BOW = vectorizer.fit_transform(X_train)
    X_test_BOW = vectorizer.transform(X_test)
    return X_train_BOW, X_test_BOW, vectorizer


extract_features(X_train, X_test)

NameError: name 'max_features' is not defined

# Train SVM with hyperparameter tuning

In [ ]:
# Grid search function for threshold optimization and SVM training
def grid_search_and_train_svm(X_train, X_validation, y_train, y_validation, max_features, ngram_range, cv=5, scoring='accuracy'):
    df_stop_words = stop_word_frequencies(X_train)

    best_threshold = None
    best_model = None
    best_accuracy = 0

    thresholds = np.arange(0.10, 1.10, 0.10)

    for threshold in thresholds:
        stop_words = filtered_stop_words(df_stop_words, threshold)

        X_train_tfidf, X_validation_tfidf, vectorizer = extract_features_gs(X_train, X_validation, stop_words, max_features, ngram_range)

        # Train SVM with Grid Search for hyperparameter tuning
        param_grid = {
            'C': [0.1, 1, 10],
            'kernel': ['linear'],
            'class_weight': ['balanced', None]
        }
        grid_search = GridSearchCV(SVC(), param_grid, cv=cv, n_jobs=-1)
        grid_search.fit(X_train_tfidf, y_train)
        best_model = grid_search.best_estimator_

        y_validation_pred = best_model.predict(X_validation_tfidf)
        accuracy = accuracy_score(y_validation, y_validation_pred)

        print(f"Threshold: {threshold:.2f}, Validation Accuracy: {accuracy:.4f}, Best SVM Parameters: {grid_search.best_params_}")

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_threshold = threshold

    print(f"Best Threshold: {best_threshold:.2f}, Best Validation Accuracy: {best_accuracy:.4f}")

    return best_model, best_threshold, vectorizer


In [ ]:
def train_svm(X_train_tfidf, y_train):
    param_grid = {
        'C': [0.1, 1, 10],
        'kernel': ['linear'],
        'class_weight': ['balanced', None]
    }
    grid_search = GridSearchCV(SVC(), param_grid, cv=5, n_jobs=-1)
    grid_search.fit(X_train_tfidf, y_train)
    best_model = grid_search.best_estimator_
    print(f"Best SVM Parameters: {grid_search.best_params_}")
    return best_model


# Evaluate the model

In [ ]:
def evaluate_model(model, X_test_tfidf, y_test):
    predictions = model.predict(X_test_tfidf)
    accuracy = accuracy_score(y_test, predictions)
    report = classification_report(y_test, predictions)
    cm = confusion_matrix(y_test, predictions)
    return accuracy, cm, report


# Save the best model

In [ ]:
def save_model(model, vectorizer, label_encoder, model_path, vectorizer_path, label_encoder_path):
    dump(model, model_path)
    dump(vectorizer, vectorizer_path)
    dump(label_encoder, label_encoder_path)


# Load the model and vectorizer

In [ ]:
def load_model(model_path, vectorizer_path, label_encoder_path):
    model = load(model_path)
    vectorizer = load(vectorizer_path)
    label_encoder = load(label_encoder_path)
    return model, vectorizer, label_encoder


# Predict the genre for new movie plots

In [ ]:
def predict_genre(model, vectorizer, label_encoder, input_file, output_file, lemmatize, combine_fields):
    data = read_csv(input_file, sep='\t', names=['title', 'from', 'director', 'plot'])
    data = preprocess_data(data, lemmatize, combine_fields)
    plots = data['combined_text']
    plot_vectors = vectorizer.transform(plots)
    predicted_genres_encoded = model.predict(plot_vectors)
    predicted_genres = label_encoder.inverse_transform(predicted_genres_encoded)

    # Save the results to a file
    data['genre'] = predicted_genres
    data[['title', 'from', 'director', 'plot', 'genre']].to_csv(output_file, sep='\t', index=False)


# Main function to run the program

In [ ]:
data

,plot,from,director,title,genre,combined_text


In [ ]:
if __name__ == '__main__':

    train_filepath = 'train.txt'
    #test_filepath = 'test_no_labels.txt'
    #results_filepath = 'results.txt'
    max_features = 10000
    ngram_range = '1,3'
    lemma = False
    combine_fields = 'from,director,title'

    # Parse ngram_range as a tuple
    ngram_range = tuple(map(int, ngram_range.split(',')))

    # Parse combine_fields as a list of field names
    combine_fields = combine_fields.split(',')

    """# Convert the stop words string into a list (if not using 'english')
    if stop_words != 'english':
        stop_words = stop_words.split(',')
    else:
        stop_words = 'english'"""

    # Load and preprocess the data
    data = load_data(train_filepath)
    data = preprocess_data(data, lemma, combine_fields)

    # Analyze class distribution
    print("Class distribution:")
    print(data['genre'].value_counts())

    # Split the data into training and test sets
    X_train, X_validation, X_test, y_train, y_validation, y_test, label_encoder = split_data(data)

    # Train SVM with hyperparameter tuning
    best_svm_model, best_threshold, vectorizer = grid_search_and_train_svm(X_train, X_validation, y_train, y_validation, max_features, ngram_range, cv=5, scoring='accuracy')



Class distribution:
genre
drama        1676
comedy       1193
horror       1108
action       1059
romance       886
western       829
crime         541
animation     535
sci-fi        214
Name: count, dtype: int64
Threshold: 0.10, Validation Accuracy: 0.7113, Best SVM Parameters: {'C': 1, 'class_weight': 'balanced', 'kernel': 'linear'}
Threshold: 0.20, Validation Accuracy: 0.7141, Best SVM Parameters: {'C': 1, 'class_weight': 'balanced', 'kernel': 'linear'}
Threshold: 0.30, Validation Accuracy: 0.7169, Best SVM Parameters: {'C': 1, 'class_weight': 'balanced', 'kernel': 'linear'}
Threshold: 0.40, Validation Accuracy: 0.7182, Best SVM Parameters: {'C': 1, 'class_weight': 'balanced', 'kernel': 'linear'}
Threshold: 0.50, Validation Accuracy: 0.7141, Best SVM Parameters: {'C': 1, 'class_weight': 'balanced', 'kernel': 'linear'}
Threshold: 0.60, Validation Accuracy: 0.7072, Best SVM Parameters: {'C': 1, 'class_weight': 'balanced', 'kernel': 'linear'}


In [ ]:
  # Evaluate model
    svm_accuracy, svm_cm, svm_report = evaluate_model(best_svm_model, X_test_tfidf, y_test)
    print(f"Optimized SVM Accuracy: {svm_accuracy * 100:.2f}%")
    print("Classification Report:")
    print(svm_report)

    # Cross-validation
    cv_scores = cross_val_score(best_svm_model, X_train_tfidf, y_train, cv=5, scoring='accuracy')
    print(f"Cross-validation Accuracy: {cv_scores.mean() * 100:.2f}% (+/- {cv_scores.std() * 100:.2f}%)")

    # Save the best model
    save_model(best_svm_model, vectorizer, label_encoder, 'best_model.pkl', 'vectorizer.pkl', 'label_encoder.pkl')

    # Predict genres for the test file and save results
    predict_genre(best_svm_model, vectorizer, label_encoder, test_filepath, results_filepath, lemma, combine_fields)


In [ ]:
def stop_words_frequency(data):
  # CountVectorizer to create a document-term matrix for stop words
  stop_words = set(stopwords.words('english'))  # English stop words
  vectorizer = CountVectorizer(vocabulary=stop_words, binary=True)  # Count presence
  X = vectorizer.fit_transform(data['title_plot'])  # Transform the title-plot feature

  # Compute document frequencies
  doc_freq = X.sum(axis=0) / X.shape[0]  # Fraction of documents (observations) containing each stop word

  # Sort stop words by document frequency.
  stop_word_df = pd.DataFrame({
      'stop_word': vectorizer.get_feature_names_out(),
      'doc_frequency': doc_freq.A1
  }).sort_values(by='doc_frequency', ascending=False)

  return stop_word_df

def list_stop_words(df_stop_words, threshold):
  # Filter stop words by a document frequency threshold
  top_stop_words = df_stop_words[df_stop_words['doc_frequency'] >= threshold]
  return list(top_stop_words['stop_word'].values)

# Function to remove stop words
def filter_tokens(tokens, top_stop_words):
    return [word for word in tokens if word not in top_stop_words]

def remove_stop_words(data, top_stop_words):
  return data['words'].apply(filter_tokens, args=(top_stop_words,))

In [ ]:
stop_words = list_stop_words(stop_word_df, 0.5)
remove_stop_words(X_train, stop_words)

KeyError: 'words'

In [ ]:
stop_word_df = stop_words_frequency(data)
stop_word_df.head()

,stop_word,doc_frequency
129,the,0.976993
141,to,0.948016
10,and,0.942420
17,be,0.940182
98,of,0.925631
